# Workflow: 

- Step 1.  For 'Position_clean' column, find all rows that contain substring combo ('intern' OR 'co-op) AND ('data science' OR 'data scientist' OR 'ML' or 'machine learning') *case insensitive*
    - replace all with 'Data Scientist Intern' 
- Step 2.  find all rows that contain substring 'data science'*case insensitive*
    - replace case by case with 'xx Data Scientist' 
- Step 3. find all rows that contain substring 'analytics' *case insensitive*
    - replace case by case with 'xx Data Scientist' 
- Step 4. find all rows that contain substring 'ML' or 'machine learning' (assume: exclude 'AI' roles) *case-insensitive*
    - replace case by case with 'xx Data Scientist' 
    - e.g. For Canada: 
            - "Research Scientist, Core ML" -> "Senior Data Scientist" 
            - 'Senior Machine Learning Scientist" -> "Senior Data Scientist" 
            - "Senior Machine Learning Scientist, Natural Language Processing" -> "Senior Data Scientist" 
            - "Sr ML/AI Data Analyst" (4 positions) -> "Data Scientist"
 
- Step 5. Other special cases (from examining df.Position. unique()): replace case by case with 'xx Data Scientist' 
  - For Canada: 
       -'Programmer/Developer Lead - Sr Data Analyst - Visualization' -> "Senior Data Scientist"
                    
- Step 6: find all rows that do NOT contain substring *case-insensitive* 'Data Scientist'(e.g. 'software engineer', 'AI research scientist', 'data engineer', 'software developer', 'database developer', 'data analyst') -> replace with np.nan (for easier removal of these rows later) 


[see here for str.replace](https://stackoverflow.com/questions/39768547/replace-whole-string-if-it-contains-substring-in-pandas)

**note: order matters! e.g. '*Senior* *director*, data scientist' - need to change to LDS, before SR**

- Step 7A. replace any substrings 'lead' or 'director' or 'manager' or 'principal' -> 'Lead Data Scientist'
- Step 7B. replace any substrings 'sr' or 'senior' -> 'Senior Data Scientist'
- Step 7C. replace any substrings jr' or 'junior' -> 'Junior Data Scientist'
- Step 7D. replace substrings NOT 'intern' NOT 'junior' NOT 'senior' NOT 'lead' NOT NaN  -> 'Data Scientist' 


AFTER merging with Sasha's cleaned columns, do following: 
- drop rows that have NaN in the 'Position_clean' column [see how here](https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan)
- check and drop duplicates
- drop other rows with certain NaN fields

In [1]:
import pandas as pd
import numpy as np

In [2]:
INTERN = 'Data Scientist Intern'
JR = 'Junior Data Scientist'
SR = 'Senior Data Scientist'
DS = 'Data Scientist'
LDS = 'Lead Data Scientist'
rm = np.nan

In [3]:
df_us = pd.read_csv('elisa_us_new.csv', index_col = 0)

In [4]:
df_us.Position.unique() 

array(['Jr. Data Scientist', 'Data Scientist-Junior', 'Data Analyst',
       'Scientist III, Pharma R & D', 'Database Developer',
       'Senior Scientist I', 'Energy Research and Data Analyst',
       'Senior, Measurement Data Analyst', 'Associate Data Analyst',
       'Principal Quality Scientist', 'Senior Database Developer',
       'Data Scientist', 'Population Health Data Analyst',
       'Care Coordination Quality Improvement Data Analyst',
       'Web Data Analyst (South of Milwaukee)', 'Enterprise Data Analyst',
       'Financial Data Analyst', 'Contract Data Analyst',
       'Senior Scientist in Product Development', 'Junior Data Scientist',
       'Senior Data Scientist Advisor', 'Senior SQL Database Developer',
       'Sales Data Analyst (Outdoor/Hunting Industry)',
       'Operations Data Analyst', 'DEI Data Analyst',
       'Finance Data Analyst Internship - Paid (South of Milwaukee)',
       'SQL Data Analyst', 'Energy Market Data Analyst',
       'Healthcare Data Analyst

In [5]:
# duplicate
df_us['Position_clean'] = df_us.loc[:, 'Position']

In [6]:
df_us[df_us['Position'] == 'Lead Account Data Scientist']

,Position,Description,Company,Location,Glassdoor Salary Estimate,Rating,Size,Revenue,Type,Year Founded,Industry,Sector,Position_clean
148,Lead Account Data Scientist,Expectations\n\n\nHard skills:\n2-3 years in a...,Bell Integrator,"San Jose, CA",NF,3.4,1001 to 5000 Employees,Unknown / Non-Applicable,Company - Private,2003,IT Services,Information Technology,Lead Account Data Scientist


In [7]:
## step 1
s = ['intern', 'co-op', 'co op', 'placem']
cond_or1 = '|'.join(s)
t = ["data scien", "machine learn"]
cond_or2 = '|'.join(t)
bool_intern = (df_us['Position'].str.contains(cond_or1, case = False))& \
               ( (df_us['Position'].str.contains(cond_or2, case = False)) |\
                (df_us['Position'].str.contains('ML', case = True))
               ) 
df_us[bool_intern] # display rows

,Position,Description,Company,Location,Glassdoor Salary Estimate,Rating,Size,Revenue,Type,Year Founded,Industry,Sector,Position_clean
53,"Data Scientist, Intern",Help us shape the future of education.\n\nOur ...,Duolingo,"Pittsburgh, PA",NF,4.9,201 to 500 Employees,$10 to $25 million (USD),Company - Private,2011,Internet,Information Technology,"Data Scientist, Intern"
75,Data Scientist Intern,Intern - Data Science\n\nThe New York Times is...,The New York Times,"New York, NY",NF,3.2,1 to 50 Employees,Less than $1 million (USD),Company - Private,NF,NF,NF,Data Scientist Intern
81,Data Scientist Intern,Internet Brands is looking for a Data Scientis...,Internet Brands,"Los Angeles, CA",NF,3.3,1001 to 5000 Employees,$1 to $2 billion (USD),Company - Private,1998,Internet,Information Technology,Data Scientist Intern
89,Data Scientist Intern,"Joby Overview: Located in Northern California,...",Joby Aviation,"San Carlos, CA",NF,4.0,51 to 200 Employees,Less than $1 million (USD),Company - Private,NF,NF,NF,Data Scientist Intern
97,"Data Scientist, Intern",Help us shape the future of education.\n\nOur ...,Duolingo,"Pittsburgh, PA",NF,4.9,201 to 500 Employees,$10 to $25 million (USD),Company - Private,2011,Internet,Information Technology,"Data Scientist, Intern"
118,Data Scientist Intern,Worker Sub-Type:\n\nStudent (Fixed Term)\n\nJo...,Blackberry,"Irving, TX",NF,3.3,1001 to 5000 Employees,$5 to $10 billion (USD),Company - Public,1984,Enterprise Software & Network Solutions,Information Technology,Data Scientist Intern
123,Data Scientist Intern,Bayer is a global enterprise with core compete...,Bayer,"Creve Coeur, MO",NF,4.1,5001 to 10000 Employees,$10+ billion (USD),Company - Private,1863,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,Data Scientist Intern
132,Data Scientist Intern,** Elutions will be unable to sponsor or trans...,Elutions,"Tampa, FL",NF,4.0,1 to 50 Employees,$1 to $5 million (USD),Company - Private,1994,Computer Hardware & Software,Information Technology,Data Scientist Intern
140,Data Scientist Intern,Build the Future\n\nDo you enjoy testing the l...,McGraw Hill,United States,NF,3.6,1001 to 5000 Employees,$1 to $2 billion (USD),Company - Private,1888,Education Training Services,Education,Data Scientist Intern
145,Data Scientist Intern,HENKEL IS FOR THOSE WHO STEP UP. DO YOU?\nAt H...,Henkel,"Irvine, CA",NF,4.3,10000+ Employees,$10+ billion (USD),Company - Public,1876,Consumer Products Manufacturing,Manufacturing,Data Scientist Intern


In [8]:
df_us.loc[bool_intern, 'Position_clean'] = INTERN
# 1c. check by comparing 2 columns
df_us[bool_intern][['Position', 'Position_clean']]

,Position,Position_clean
53,"Data Scientist, Intern",Data Scientist Intern
75,Data Scientist Intern,Data Scientist Intern
81,Data Scientist Intern,Data Scientist Intern
89,Data Scientist Intern,Data Scientist Intern
97,"Data Scientist, Intern",Data Scientist Intern
118,Data Scientist Intern,Data Scientist Intern
123,Data Scientist Intern,Data Scientist Intern
132,Data Scientist Intern,Data Scientist Intern
140,Data Scientist Intern,Data Scientist Intern
145,Data Scientist Intern,Data Scientist Intern


In [9]:
## Step 2
bool_2 = df_us['Position'].str.contains('data science', case = False)
df_us[bool_2] # display

,Position,Description,Company,Location,Glassdoor Salary Estimate,Rating,Size,Revenue,Type,Year Founded,Industry,Sector,Position_clean
136,Data Analyst I - Data Science Team,"Wednesday, October 14, 2020\n\nThe Data Scienc...",CNA Corporation,"Arlington, VA",$78k-$129k (Glassdoor Est.),3.3,501 to 1000 Employees,$100 to $500 million (USD),Nonprofit Organization,1942,Aerospace & Defense,Aerospace & Defense,Data Analyst I - Data Science Team
137,Junior Data Scientist Data Science Development...,Do you want to be part of a team that encourag...,MassMutual,"Amherst, MA",$73k-$124k (Glassdoor Est.),3.8,5001 to 10000 Employees,$10+ billion (USD),Company - Private,1851,Insurance Carriers,Insurance,Junior Data Scientist Data Science Development...
145,"VP, Data Science",Oversee a team of Data Scientists with overall...,PennyMac,"Phoenix, AZ",$180k-$286k (Glassdoor Est.),4.2,1001 to 5000 Employees,$500 million to $1 billion (USD),Company - Public,2008,Lending,Finance,"VP, Data Science"
173,"VP, Data Science",Oversee a team of Data Scientists with overall...,PennyMac,"Phoenix, AZ",$180k-$286k (Glassdoor Est.),4.2,1001 to 5000 Employees,$500 million to $1 billion (USD),Company - Public,2008,Lending,Finance,"VP, Data Science"
208,"Translational Data Scientist, Oncology Data Sc...","18-Sep-2020\n\nJob ID\n303124BR\n{""QuestionNam...",Novartis,"Cambridge, MA",$86k-$143k (Glassdoor Est.),4.1,10000+ Employees,$10+ billion (USD),Company - Public,1996,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,"Translational Data Scientist, Oncology Data Sc..."
11,Data Science & Analytics Manager,If you ever wanted to be on the ground floor o...,Takeoff Technologies,"Boston, MA",NF,4.3,201 to 500 Employees,Unknown / Non-Applicable,Company - Private,2016,Computer Hardware & Software,Information Technology,Data Science & Analytics Manager
98,Senior Manager of Data Science - Operations,Guild is hiring a Senior Manager of Data Scien...,Guild Education,"Denver, CO",$139k-$219k (Glassdoor Est.),3.6,501 to 1000 Employees,Unknown / Non-Applicable,Company - Private,2015,Enterprise Software & Network Solutions,Information Technology,Senior Manager of Data Science - Operations
107,"Translational Data Scientist, Oncology Data Sc...","18-Sep-2020\n\nJob ID\n303124BR\n{""QuestionNam...",Novartis,"Cambridge, MA",$86k-$143k (Glassdoor Est.),4.1,10000+ Employees,$10+ billion (USD),Company - Public,1996,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,"Translational Data Scientist, Oncology Data Sc..."
135,Data Science & Analytics Manager,If you ever wanted to be on the ground floor o...,Takeoff Technologies,"Boston, MA",NF,4.3,201 to 500 Employees,Unknown / Non-Applicable,Company - Private,2016,Computer Hardware & Software,Information Technology,Data Science & Analytics Manager
89,Senior/Staff Data Science Engineer,Are you an experience data engineer/ scientist...,Brightline,Remote,NF,5.0,1 to 50 Employees,Less than $1 million (USD),Company - Private,2019,NF,NF,Senior/Staff Data Science Engineer


In [10]:
# special case 1 
df_us.loc[df_us['Position'] == 'Data Science Support Analyst', 'Position_clean'] = rm
df_us.loc[df_us['Position'] == 'Data Science Instructor, Part-Time - REMOTE', 'Position_clean'] = rm

In [11]:
# special case 2
df_us.loc[df_us['Position'] == 'Data Science', 'Position_clean'] = DS
# check 
#df_us[df_us['Position'] == 'Data Science']

In [12]:
# jr ds
bool_jr = (df_us['Position_clean'].str.contains('junior|jr|recent grad|Analyst I', case = False)) & \
           (df_us['Position_clean'].str.contains('data science', case = False))
df_us.loc[bool_jr, 'Position_clean'] = JR
# lead ds
bool_lds = (df_us['Position_clean'].str.contains('VP|head|manager|director|lead|principal', case = False)) & \
           (df_us['Position_clean'].str.contains('data science', case = False))
df_us.loc[bool_lds, 'Position_clean'] = LDS
# sr ds
bool_sr = (df_us['Position_clean'].str.contains('senior/staff|fellowship|senior|sr', case = False)) & \
           (df_us['Position_clean'].str.contains('data science', case = False))
df_us.loc[bool_sr, 'Position_clean'] = SR
# ds
bool_ds = (df_us['Position_clean'].str.contains('software engineer|Scientist II', case = False)) & \
           (df_us['Position_clean'].str.contains('data science', case = False))
df_us.loc[bool_ds, 'Position_clean'] = DS

In [13]:
# check 
df_us[bool_2][['Position', 'Position_clean']]

,Position,Position_clean
136,Data Analyst I - Data Science Team,Junior Data Scientist
137,Junior Data Scientist Data Science Development...,Junior Data Scientist
145,"VP, Data Science",Lead Data Scientist
173,"VP, Data Science",Lead Data Scientist
208,"Translational Data Scientist, Oncology Data Sc...","Translational Data Scientist, Oncology Data Sc..."
11,Data Science & Analytics Manager,Lead Data Scientist
98,Senior Manager of Data Science - Operations,Lead Data Scientist
107,"Translational Data Scientist, Oncology Data Sc...","Translational Data Scientist, Oncology Data Sc..."
135,Data Science & Analytics Manager,Lead Data Scientist
89,Senior/Staff Data Science Engineer,Senior Data Scientist


In [14]:
## step 3: 'analytics'
bool_3 = df_us['Position'].str.contains('analytics', case = False)
#df_us[bool_3]['Position'].iloc[0:] # display

In [15]:
# special 1: remove analytics engineer
bool_eng = df_us['Position_clean'].str.contains('analytics', case = False, na = False) &\
           df_us['Position_clean'].str.contains('engineer', case = False, na = False)
df_us.loc[bool_eng, 'Position_clean'] = rm
# special 2: remove irrelevant
df_us.loc[df_us['Position'] == 'Senior Product Manager - Threat Detection and Analytics - Secureworks - Remote, US', 'Position_clean'] = rm
df_us.loc[df_us['Position'] == 'Sign Out Manager-Data Analytics & Results Virtual Hiring Event' , 'Position_clean'] = rm
df_us.loc[df_us['Position'] == 'Data Analyst, Performance Analytics', 'Position_clean'] = rm
# special 3
df_us.loc[df_us['Position'] == 'Scientist 2, Data Analytics', 'Position_clean'] = DS
df_us.loc[df_us['Position'] == 'Aon Campus - Health Analytics Data Analyst', 'Position_clean'] = DS

In [16]:
# jr ds
bool_jr = (df_us['Position_clean'].str.contains('junior|jr', case = False, na = False)) & \
           (df_us['Position_clean'].str.contains('analytics', case = False, na = False))
df_us.loc[bool_jr, 'Position_clean'] = JR
# lead ds
bool_lds = (df_us['Position_clean'].str.contains('VP|head|manag|director|lead|principal', case = False, na = False)) & \
           (df_us['Position_clean'].str.contains('analytics', case = False, na = False))
df_us.loc[bool_lds, 'Position_clean'] = LDS
# sr ds
bool_sr = (df_us['Position_clean'].str.contains('senior|sr|advance', case = False, na = False)) & \
           (df_us['Position_clean'].str.contains('analytics', case = False, na = False))
df_us.loc[bool_sr, 'Position_clean'] = SR
# ds
bool_ds = df_us['Position_clean'].str.contains('analytics', case = False, na = False)
df_us.loc[bool_ds, 'Position_clean'] = DS

In [17]:
# check 
df_us[bool_3][['Position', 'Position_clean']].iloc[40:60,]

,Position,Position_clean
156,CIB Wholesale Payments Data and Analytics - Da...,Lead Data Scientist
158,Lead Data Scientist - Analytics,Lead Data Scientist
160,"Data Scientist, Analytics-Remote Presence",Data Scientist
178,Enterprise Analytics Data Scientist Sr.,Senior Data Scientist
18,Risk Analytics Manager,Lead Data Scientist
42,Sign Out Manager-Data Analytics & Results Virt...,NaN
80,Business Analytics Manager,Lead Data Scientist
105,Corporate Analytics Manager (South of Milwaukee),Lead Data Scientist
112,"Data Analyst, Performance Analytics",NaN
163,"Data Scientist, Business Analytics",Data Scientist


In [18]:
## step 4 - omit since all relevant ML roles also have 'data scientist' in their titles so can deal w it in Step 7
## Step 5  - other special cases
# spec 1
bool_s = df_us['Position_clean'].str.contains('statistician|data mining|python', case = False) &\
        df_us['Position_clean'].str.contains('data analyst', case = False)
#df_us[bool_s]
df_us.loc[bool_s, 'Position_clean'] = JR
# spec 2
df_us.loc[df_us['Position'] == 'Data & Applied Scientist', 'Position_clean'] = DS
df_us.loc[df_us['Position'] == 'Sr. Data & Applied Scientist', 'Position_clean'] = SR
df_us.loc[df_us['Position'] == 'Quantitative Analyst and Risk Modeler', 'Position_clean'] = DS

In [19]:
## Step 6 - replace all non-DS with NaN
bool_notDS = df_us['Position_clean'].str.contains('Data Scientist', case = False) == False
df_us.loc[bool_notDS, 'Position_clean'] = rm
# check 
# a = df_us[df_us['Position'].str.contains('Data Scientist', case = False) == False]
# a[['Position', 'Position_clean']].shape 

In [20]:
# check no. of unique data scientist listings left after removing irrelevant rows
df_us[df_us['Position_clean'].str.contains('Data Scientist', case = False) == True]['Position'].unique().shape

(331,)

In [31]:
df_us['Position'].unique().shape

(793,)

## Out of 793 *unique* U.S. listings, get 331 *unique* data scientist (includes: 8 intern, 29 Jr., 101 Sr., 100 Lead, 93 data scientist) listings

In [21]:
## Step 7 
# 7A
# special
df_us.loc[df_us['Position'] == 'Data Scientist to Data Scientist Lead', 'Position_clean'] = DS
df_us.loc[df_us['Position'] == 'Lead Account Data Scientist', 'Position_clean'] = DS
# replace w LDS
bool_lds = df_us['Position_clean'].str.contains('lead|principal|manage|direct|VP|head', case = False, na = False)
df_us[bool_lds]['Position'].unique().shape # 100 unique LDS roles
df_us.loc[bool_lds, 'Position_clean'] = LDS

In [23]:
# 7B
bool_sr = df_us['Position_clean'].str.contains('senior|sr|expert|staff|st 3|st III|st II|clinical|intermediate|advance', case = False, na = False)
#df_us[bool_sr]['Position'].unique().shape # 101 unqiue SR roles
df_us.loc[bool_sr, 'Position_clean'] = SR
#check 
df_us[bool_sr][['Position', 'Position_clean']]

,Position,Position_clean
26,Senior Data Scientist Advisor,Senior Data Scientist
57,Sr. Data Scientist,Senior Data Scientist
61,Senior Engineer/Data Scientist - Industry 4.0,Senior Data Scientist
36,Staff Data Scientist,Senior Data Scientist
112,Staff Data Scientist - Interests,Senior Data Scientist
136,Clinical Data Scientist,Senior Data Scientist
155,Clinical Data Scientist,Senior Data Scientist
68,Staff Data Scientist - Interests,Senior Data Scientist
98,Staff Data Scientist - Interests,Senior Data Scientist
128,Staff Data Scientist - Interests,Senior Data Scientist


In [25]:
# 7C
bool_jr = df_us['Position_clean'].str.contains('junior|jr|early|entry|new grad|st 2|st 1', case = False, na = False) 
#df_us[bool_jr]
df_us[bool_jr]['Position'].unique().shape 
df_us.loc[bool_jr, 'Position_clean'] = JR

In [26]:
# 7D
bool_remain = (df_us['Position_clean'].str.contains('senior', case = False, na = False) == False) &\
                (df_us['Position_clean'].str.contains('junior', case = False, na = False) == False) &\
                (df_us['Position_clean'].str.contains('lead', case = False, na = False) == False) &\
                (df_us['Position_clean'].str.contains('intern', case = False, na = False) == False) &\
                df_us['Position_clean'].notna()
df_us.loc[bool_remain, 'Position_clean'] = DS

In [30]:
# Final step: check 
df_us[['Position', 'Position_clean']].iloc[300:400,]

# check special cases (handwritten * 2)
df_us[df_us['Position'] == 'Senior AI/ML/NLP Data Scientist']

,Position,Description,Company,Location,Glassdoor Salary Estimate,Rating,Size,Revenue,Type,Year Founded,Industry,Sector,Position_clean
108,Senior AI/ML/NLP Data Scientist,Ayata is developing its Prescriptive Analytics...,Ayata,"Dallas, TX",NF,2.4,1 to 50 Employees,Less than $1 million (USD),Company - Private,NF,Accounting,Accounting & Legal,Senior Data Scientist
117,Senior AI/ML/NLP Data Scientist,Ayata is developing its Prescriptive Analytics...,Ayata,"Dallas, TX",NF,2.4,1 to 50 Employees,Less than $1 million (USD),Company - Private,NF,Accounting,Accounting & Legal,Senior Data Scientist


In [50]:
# check 
df_us['Position_clean'].unique()

array(['Junior Data Scientist', nan, 'Data Scientist',
       'Senior Data Scientist', 'Lead Data Scientist',
       'Data Scientist Intern'], dtype=object)

In [51]:
# write to csv
df_us.to_csv('elisa_position_us.csv')

In [53]:
# read in to check 
df = pd.read_csv('elisa_position_us.csv', index_col = 0)
df.shape

(2717, 13)